### Convalution_classify_CIFAR10

David 9回顾去年著名论文All you need is a good init，当时提出了一种新型初始化权重的方法，号称在Cifar-10上达到94.16%的精度，这里用keras还原一下试试。

运用tensorflow和高级api——keras，构建一个卷积神经网络模型。应用到数据集CIFAR10，作图片类别识别任务。

### import

先导入需要的包，包括常用数据处理和图表显示模块。  
需要下载初始化权重的包：git clone https://github.com/ducha-aiki/LSUV-keras  
可惜这个包里面有错误，需要略微修改。  
66至68行替换为如下代码：  
```
layer.set_weights(weights_and_biases)
weights_and_biases[1] /= np.sqrt(variance) / np.sqrt(needed_variance)
layer.set_weights(weights_and_biases)
```

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,datasets,utils,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D 
import numpy as np
from tensorflow.keras.utils import to_categorical
from lsuv_init import LSUVinit

import matplotlib.pyplot as plt
%matplotlib inline

D:\Anacanda\envs\tensorflow1.14.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anacanda\envs\tensorflow1.14.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anacanda\envs\tensorflow1.14.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anacanda\envs\tensorflow1.14.0\lib\site-packages\tensorflow\python\framework

### 数据加载

设定一下batch大小，类别数，训练轮数，还有就是启用数据增强。

In [32]:
batch_size = 32 
num_classes = 10
epochs = 120
data_augmentation = True

使用的是keras内置加载数据。  
由于数据源是国外多伦多大学网站，这一步要事先下载好放到指定位置：  
windows：user/.keras/datasets/  
linux：~/.keras/datasets/  
并且对压缩包改名：  
cifar-10-python.tar.gz -> cifar-10-batch.tar.gz

In [34]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

对数据归一化、独热编码化

In [35]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

### 创建model

这是一个15层的CNN神经网络，具体结构很容易看明白。

In [29]:
def conv_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(48, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(48, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(GlobalMaxPooling2D())
    model.add(Dropout(0.25))
    
    model.add(Dense(500))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

### 定义损失函数

这一块没有特殊的地方，常规的损失函数定义和模型编译。

In [31]:
model = conv_model()

# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 68s 1ms/sample - loss: 2.1239 - acc: 0.3263 - val_loss: 2.0469 - val_acc: 0.4077
Epoch 2/10
50000/50000 [==============================] - 65s 1ms/sample - loss: 2.0248 - acc: 0.4310 - val_loss: 1.9941 - val_acc: 0.4641
Epoch 3/10
50000/50000 [==============================] - 71s 1ms/sample - loss: 1.9791 - acc: 0.4778 - val_loss: 1.9545 - val_acc: 0.5039
Epoch 4/10
50000/50000 [==============================] - 67s 1ms/sample - loss: 1.9479 - acc: 0.5090 - val_loss: 1.9595 - val_acc: 0.4975
Epoch 5/10
50000/50000 [==============================] - 70s 1ms/sample - loss: 1.9224 - acc: 0.5347 - val_loss: 1.9490 - val_acc: 0.5087
Epoch 6/10
50000/50000 [==============================] - 65s 1ms/sample - loss: 1.9047 - acc: 0.5531 - val_loss: 1.9063 - val_acc: 0.5516
Epoch 7/10
50000/50000 [==============================] - 67s 1ms/sample - loss: 1.8903 - acc: 0.5679 - val_loss: 1.

### 特殊初始化

论文的重点特色就在于此，使用特殊的初始化方法，得以提升性能。  
第二行不重要，是监控。

In [ ]:
model = LSUVinit(model,x_train[:batch_size,:,:,:]) 
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

### 数据增强

数据增强，属于图像处理的常规手段了。  
可以选择不开启。

In [ ]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test), callbacks=[tbCallBack])

In [13]:
model.save('CIFAR10_conv_model')